# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [2]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [3]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [4]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-02-22'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2603,2023-02-16,"$24,307.35","$25,134.12","$23,602.52","$23,623.47","$23,623.47",39316664596
2604,2023-02-17,"$23,621.28","$24,924.04","$23,460.76","$24,565.60","$24,565.60",41358451255
2605,2023-02-18,"$24,565.30","$24,798.84","$24,468.37","$24,641.28","$24,641.28",19625427158
2606,2023-02-19,"$24,640.03","$25,093.05","$24,327.64","$24,327.64","$24,327.64",25555105670
2607,2023-02-20,"$24,336.62","$25,020.46","$23,927.91","$24,829.15","$24,829.15",28987376573


In [8]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [28]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {"Date": "ds", "Open": "y"}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names)
df

,ds,y
0,2016-01-01,$430.72
1,2016-01-02,$434.62
2,2016-01-03,$433.58
3,2016-01-04,$430.06
4,2016-01-05,$433.07
...,...,...
2603,2023-02-16,"$24,307.35"
2604,2023-02-17,"$23,621.28"
2605,2023-02-18,"$24,565.30"
2606,2023-02-19,"$24,640.03"


In [29]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [30]:
df.tail()

,ds,y
2603,2023-02-16,"$24,307.35"
2604,2023-02-17,"$23,621.28"
2605,2023-02-18,"$24,565.30"
2606,2023-02-19,"$24,640.03"
2607,2023-02-20,"$24,336.62"


In [45]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [46]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')

In [47]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [48]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2968,2024-02-16
2969,2024-02-17
2970,2024-02-18
2971,2024-02-19
2972,2024-02-20


In [54]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
3,2016-01-04
4,2016-01-05
5,2016-01-06
6,2016-01-07
...,...
2966,2024-02-14
2967,2024-02-15
2968,2024-02-16
2971,2024-02-19


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [50]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [51]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2118,2024-02-14,"$-17,642.36","$-38,351.98","$3,873.93"
2119,2024-02-15,"$-17,874.97","$-38,663.15","$3,572.87"
2120,2024-02-16,"$-17,987.71","$-38,406.27","$4,888.38"
2121,2024-02-19,"$-18,411.89","$-39,568.19","$4,015.78"
2122,2024-02-20,"$-18,494.13","$-39,368.63","$5,027.82"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [52]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [53]:
plot_components_plotly(m, forecast)